In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, classification_report, plot_roc_curve
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

import plotly.express as px
sns.set_theme(style='darkgrid')


In [ ]:
df = pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()


# **EDA (Exploratory Data Analysis)**

In [ ]:
# check for how many womens are prone to heart-attack
women_stroke = df.loc[df.sex == 0]['output']
women_stroke_percentage = sum(women_stroke)/len(women_stroke)
print('The % of womens prone to heart-attack: {}%'.format(women_stroke_percentage*100))

Surprisingly there are 75% of the women are prone to heart-attack

In [ ]:
women = df[df['sex']==0]
men = df[df['sex']==1]


Check for the following relationship;
* cp
* trtbps
* chol
* restecg
* thalachh
* oldpeak

In [ ]:
def cp_density(women, men):
    plt.figure(figsize=(16,6))
    sns.kdeplot(women['cp'], label = 'Women cp values', shade = True, color='red')
    sns.kdeplot(men['cp'], label = 'men cp values', shade = True, color = 'blue')
    plt.legend()
    plt.title('Cp values comparision', weight='bold')
    plt.xlabel('Cp values')
    
def resting_blood_pressure_comparision(women, men):
    plt.figure(figsize=(16,6))
    sns.kdeplot(women['trtbps'], label = 'Women rbp values', shade = True, color='red')
    sns.kdeplot(men['trtbps'], label = 'men rbp values', shade = True, color = 'blue')
    plt.title('Resting blood pressure values comparision', weight='bold')
    plt.legend()
    plt.xlabel('trtbps values')
    
def cholestarol(women, men):
    plt.figure(figsize=(16,6))
    sns.kdeplot(women['chol'], label = 'Women cholestoral values', shade = True, color='green')
    sns.kdeplot(men['chol'], label = 'men cholestoral values', shade = True, color = 'black')
    plt.title('Cholestarol values comparision', weight='bold')
    plt.legend()
    plt.xlabel('Cholestral values')
    
def resting_electrocardiographic(women, men):
    plt.figure(figsize=(16,6))
    sns.kdeplot(women['restecg'], label = 'Women resting electrocardiographic values', shade = True, color='green')
    sns.kdeplot(men['restecg'], label = 'men resting electrocardiographic values', shade = True, color = 'black')
    plt.title('resting electrocardiographic values comparision', weight='bold')
    plt.legend()
    plt.xlabel('resting electrocardiographic values')

def maximum_heartrate_achieved(women, men):
    plt.figure(figsize=(16,6))
    sns.kdeplot(women['thalachh'], label = 'Women cp values', shade = True, color='black')
    sns.kdeplot(men['thalachh'], label = 'men cp values', shade = True, color = 'yellow')
    plt.title('Maximum heart-rate achieved values comparision', weight='bold')
    plt.legend()
    plt.xlabel('Maximum heart-rate achieved values')
    

In [ ]:
cp_density(women, men)

* This shows that most of the men's are free from heart-attacks.
* Both the genders show's spike for non-anginal pain. Women has the great tendency for non-anginal chest pain.

In [ ]:
resting_blood_pressure_comparision(women, men)

* Men's Resting blood pressure values has the probable density of 120.
* Women's Resting blood pressure values has the probable density of 135.

In [ ]:
cholestarol(women, men)

* As we can see here that, cholestoral values of range 200 - 270 falls majorly under womens category. 
* It's interesting that only the men's are showing values above 350.

In [ ]:
resting_electrocardiographic(women, men)

* Predominently women shows probable or definite left ventricular hypertrophy by Estes.

In [ ]:
maximum_heartrate_achieved(women, men)

There are lot of women's who have achived maximum heart-rate than men.

In [ ]:
plt.figure(figsize=(10,10))
sns.pairplot(df, hue='output')

In [ ]:
df.head()

In [ ]:
def eda_for_several_attributes(data):
    features = ['slp','thalachh','age','caa']
    for var in features:
        plt.figure(figsize=(16,6))
        g = sns.scatterplot(data = data, x = 'oldpeak', y = data[var], hue = 'output')
        plt.title(var, weight='bold')
        plt.grid(linestyle = '--', axis = 'y')
        plt.show()
            
eda_for_several_attributes(df)

* The less maximum heart-rate achieved the less chances of heart-attack.
* The slp value of 1 shows feable chances of heart-attack.

In [ ]:
# check for the age and sex
def check_the_risk(data):
    for var in ['sex','age']:
        plt.figure(figsize=(16,8))
        g = sns.countplot(data=data, x = data[var], hue = data.output, palette='OrRd')
        plt.grid(linestyle = '--', axis='y')
        plt.title(var)
        
        for i in ['top','left']:
            g.spines[i].set_visible(False)
            
check_the_risk(df)

There is a spike in for heart-attack at the age of 41, 44, 51, 52,54.

# **Correlational Map**

In [ ]:
def find_correlational_map(data):
    plt.figure(figsize=(16,12))
    sns.heatmap(data.corr(), annot=True, cmap='OrRd')
    plt.title('Correlational Map', weight='bold')
    print('---'*50)
    print(data.corr().output.sort_values(ascending = False))
    plt.tight_layout()
    
find_correlational_map(df)

# **Data Splitting**

In [ ]:
df.head()

In [ ]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [ ]:
X.shape, y.shape

In [ ]:
# Let's split the date into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_train.shape, X_test.shape

In [ ]:
y_train.shape, y_test.shape

# **Model Building**

In [ ]:
# let's create a pipline 
pipeline = make_pipeline(RobustScaler()) # creating pipeline for model building

LR = make_pipeline(pipeline, LogisticRegression(random_state=0)) # LogisticRegression pipeline
DT = make_pipeline(pipeline, DecisionTreeClassifier(random_state=0)) # DecisionTree Classifier pipeline
RF = make_pipeline(pipeline, RandomForestClassifier(random_state=0)) # RandomForest Classifier pipeline
AC = make_pipeline(pipeline, AdaBoostClassifier(random_state=0)) # Adaboost Classifier pipeline
NB = make_pipeline(pipeline, GaussianNB()) # Naive bayes pipeline
KN = make_pipeline(pipeline, KNeighborsClassifier()) # KNeighbor pipeline
SV = make_pipeline(pipeline, SVC(random_state=0)) # Support vector pipeline

In [ ]:
# creating model_dict
model_dictionary = {
    'Logistic_Regression':LR,
    'DecisionTree_Classifier':DT,
    'RandomForest_classifier':RF,
    'Adaboost_Classifier':AC,
    'Naivebayes_Classifier':NB,
    'KNeighbors_classifier':KN,
    'Support_Vector':SV
}

In [ ]:
print(model_dictionary)

In [ ]:
# define a function to fit the model and return it's accuracy, classification report and confusion matrix
def model_fitting(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print('The accuracy score of the model is: {}%'.format(accuracy_score(y_test, y_pred)* 100))
    print('-----'*20)
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

# **Selecting The Best Model**

In [ ]:
for name, model in model_dictionary.items():
    print('---'*10)
    print(name)
    model_fitting(model)

As we can see:
* Ada boost has got 90% accuracy with only 6 misclassified classes.
* It' has a precision of 0.86 for classes 0 and 0.94 for classes 1, which is better than all other algorithms.

Let's use Adaboost Model

In [ ]:
model = AdaBoostClassifier(random_state=0)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
def find_confusion_matrix(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, cmap='OrRd')
    plt.title('Confusion Matrix', weight='bold')
    print(classification_report(y_test, y_pred))
    plot_roc_curve(model, X_test, y_test)
    
    
find_confusion_matrix(y_test, y_pred)

In [ ]:
print('The accuracy of the model is: {}%'.format(round(accuracy_score(y_test, y_pred)*100, 2)))

# **THANK YOOOOUU!!!!!!!!!**